# Westminster Ground Truth Analysis with MetaShape

This notebook demonstrates the complete workflow for creating orthomosaics from DJI drone imagery using **Agisoft MetaShape** and evaluating their accuracy:

1. **Data Loading**: Load images and GCPs from CSV
2. **GCP Conversion**: Convert UTM coordinates to WGS84 lat/lon for MetaShape
3. **Orthomosaic Creation**: Generate orthomosaics with and without GCPs using MetaShape
4. **Basemap Comparison**: Download basemaps and quantify absolute accuracy
5. **Quality Report**: Generate comprehensive comparison report

## Datasets:
- **Dataset 1**: DJI_202510060955_017_25-3288 (543 images)
- **Dataset 2**: DJI_202510060955_019_25-3288 (528 images)

For each dataset, we create:
- Orthomosaic **without** GCPs
- Orthomosaic **with** GCPs

All orthomosaics are compared against reference basemaps to evaluate accuracy.


## Setup: Install Dependencies

First, install the required packages. Note: This notebook requires **Agisoft MetaShape Python API** to be installed separately.


In [ ]:
# Install required packages
import subprocess
import sys
from pathlib import Path

# Try to install from requirements.txt first
requirements_file = Path("requirements.txt")
if requirements_file.exists():
    print("Installing packages from requirements.txt...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_file)])
    print("✓ Packages installed from requirements.txt")
else:
    # Fallback: install packages individually
    print("requirements.txt not found. Installing packages individually...")
    packages = [
        "numpy>=1.24.0",
        "rasterio>=1.3.0",
        "pillow>=10.0.0",
        "matplotlib>=3.7.0",
        "pandas>=2.0.0",
        "pyproj>=3.6.0",
        "requests>=2.31.0",
        "utm>=0.7.0",
    ]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("✓ All packages installed")

# Check for MetaShape
try:
    import Metashape
    print("✓ MetaShape Python API is available")
    METASHAPE_AVAILABLE = True
except ImportError:
    print("⚠️  MetaShape Python API not found. Please install Agisoft MetaShape and its Python API.")
    print("   The API is typically installed with MetaShape at:")
    print("   - Windows: C:\\Program Files\\Agisoft\\Metashape Pro\\python")
    print("   - macOS: /Applications/Metashape Pro/Metashape.app/Contents/Frameworks/Python.framework/Versions/3.9")
    print("   - Linux: /opt/metashape-pro/lib/python3.9")
    METASHAPE_AVAILABLE = False

print("\nSetup complete!")


## Imports


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
import logging
import json
import csv
import xml.etree.ElementTree as ET
import utm
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Add package to path
sys.path.insert(0, str(Path.cwd()))

# Import Westminster-specific modules
from westminster_ground_truth_analysis import (
    GCPParser,
    download_basemap,
    compare_orthomosaic_to_basemap,
)

# Try to import MetaShape processor from qualicum_beach package
# If not available, we'll define the functions locally
try:
    from qualicum_beach_gcp_analysis import (
        process_orthomosaic,
        PhotoMatchQuality,
        DepthMapQuality,
        export_to_metashape_csv,
        export_to_metashape_xml,
    )
    print("✓ Using MetaShape processor from qualicum_beach_gcp_analysis")
    USE_QUALICUM_PACKAGE = True
except ImportError:
    print("⚠️  qualicum_beach_gcp_analysis not found. Will define MetaShape functions locally.")
    USE_QUALICUM_PACKAGE = False
    # We'll define these functions in the next cell if needed

# Set up paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

print("✓ Imports successful!")


## Step 1: Load Ground Control Points


In [ ]:
# Parse GCP file (UTM coordinates)
gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
gcp_parser = GCPParser(str(gcp_file))

gcps_utm = gcp_parser.get_gcps()
print(f"Loaded {len(gcps_utm)} ground control points (UTM format)")

# Display first few GCPs
print("\nFirst few GCPs (UTM):")
for gcp in gcps_utm[:5]:
    print(f"  {gcp.name}: X={gcp.x:.2f}, Y={gcp.y:.2f}, Z={gcp.z:.2f}")

# Get bounds
min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
print(f"\nGCP Bounds (UTM): X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")


## Step 2: Convert GCPs to WGS84 Lat/Lon for MetaShape

MetaShape expects GCPs in WGS84 lat/lon format. We need to convert from UTM Zone 10N.


In [ ]:
# Convert UTM to WGS84 lat/lon
# NOTE: In the CSV, X column is actually Northing (~5.45M) and Y column is Easting (~500k)
# utm.to_latlon expects (easting, northing), so we need to swap them
gcps_wgs84 = []

for gcp in gcps_utm:
    # Convert UTM to lat/lon (UTM Zone 10N)
    # X is Northing, Y is Easting
    lat, lon = utm.to_latlon(gcp.y, gcp.x, 10, 'N')
    
    gcp_dict = {
        'id': gcp.name,
        'label': gcp.name,
        'lat': lat,
        'lon': lon,
        'z': gcp.z,
        'accuracy': 0.1  # Default accuracy in meters
    }
    gcps_wgs84.append(gcp_dict)

print(f"Converted {len(gcps_wgs84)} GCPs to WGS84 lat/lon")
print("\nFirst few GCPs (WGS84):")
for gcp in gcps_wgs84[:5]:
    print(f"  {gcp['id']}: ({gcp['lat']:.6f}, {gcp['lon']:.6f}, z={gcp['z']:.2f})")


## Step 3: Export GCPs for MetaShape


In [ ]:
# Create output directory for GCP files
gcp_output_dir = output_dir / "gcps"
gcp_output_dir.mkdir(exist_ok=True)

# Export to MetaShape XML format (preferred by MetaShape)
if USE_QUALICUM_PACKAGE:
    gcp_xml_path = gcp_output_dir / "gcps_metashape.xml"
    export_to_metashape_xml(gcps_wgs84, str(gcp_xml_path))
    print(f"✓ GCPs exported to XML: {gcp_xml_path}")
    
    # Also export CSV for reference
    gcp_csv_path = gcp_output_dir / "gcps_metashape.csv"
    export_to_metashape_csv(gcps_wgs84, str(gcp_csv_path))
    print(f"✓ GCPs also exported to CSV: {gcp_csv_path}")
else:
    # Define export functions locally if qualicum package not available
    def export_to_metashape_xml_local(gcps, output_path):
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        root = ET.Element('document')
        chunks = ET.SubElement(root, 'chunks')
        chunk = ET.SubElement(chunks, 'chunk')
        markers = ET.SubElement(chunk, 'markers')
        
        for gcp in gcps:
            marker = ET.SubElement(markers, 'marker')
            marker.set('label', gcp.get('id', gcp.get('label', 'GCP')))
            marker.set('reference', 'true')
            
            position = ET.SubElement(marker, 'position')
            position.set('x', str(gcp.get('lon', 0.0)))
            position.set('y', str(gcp.get('lat', 0.0)))
            position.set('z', str(gcp.get('z', 0.0)))
            
            accuracy = gcp.get('accuracy', 1.0)
            accuracy_elem = ET.SubElement(marker, 'accuracy')
            accuracy_elem.set('x', str(accuracy))
            accuracy_elem.set('y', str(accuracy))
            accuracy_elem.set('z', str(accuracy))
        
        tree = ET.ElementTree(root)
        ET.indent(tree, space='  ')
        tree.write(output_path, encoding='utf-8', xml_declaration=True)
        print(f"Exported {len(gcps)} GCPs to MetaShape XML: {output_path}")
        return str(output_path)
    
    def export_to_metashape_csv_local(gcps, output_path):
        output_path = Path(output_path)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(['Label', 'X', 'Y', 'Z', 'Accuracy', 'Enabled'])
            
            for gcp in gcps:
                label = gcp.get('id', gcp.get('label', 'GCP'))
                lon = gcp.get('lon', 0.0)
                lat = gcp.get('lat', 0.0)
                z = gcp.get('z', 0.0)
                accuracy = gcp.get('accuracy', 1.0)
                writer.writerow([label, lon, lat, z, accuracy, '1'])
        
        print(f"Exported {len(gcps)} GCPs to MetaShape CSV: {output_path}")
        return str(output_path)
    
    gcp_xml_path = gcp_output_dir / "gcps_metashape.xml"
    export_to_metashape_xml_local(gcps_wgs84, str(gcp_xml_path))
    print(f"✓ GCPs exported to XML: {gcp_xml_path}")
    
    gcp_csv_path = gcp_output_dir / "gcps_metashape.csv"
    export_to_metashape_csv_local(gcps_wgs84, str(gcp_csv_path))
    print(f"✓ GCPs also exported to CSV: {gcp_csv_path}")

# Use XML file for processing (MetaShape's native format)
gcp_file_for_processing = gcp_xml_path


In [ ]:
# Define MetaShape processing functions if qualicum package not available
if not USE_QUALICUM_PACKAGE and METASHAPE_AVAILABLE:
    from enum import IntEnum
    from typing import Optional
    
    class PhotoMatchQuality(IntEnum):
        LowestQuality = 0
        LowQuality = 1
        MediumQuality = 2
        HighQuality = 4
        HighestQuality = 8
    
    class DepthMapQuality(IntEnum):
        LowestQuality = 1
        LowQuality = 2
        MediumQuality = 4
        HighQuality = 8
        UltraQuality = 16
    
    def process_orthomosaic(
        photos_dir: Path,
        output_path: Path,
        project_path: Path,
        gcp_file: Optional[Path] = None,
        product_id: str = "orthomosaic",
        clean_intermediate_files: bool = False,
        photo_match_quality: int = PhotoMatchQuality.MediumQuality,
        depth_map_quality: int = DepthMapQuality.MediumQuality,
        tiepoint_limit: int = 10000,
        use_gcps: bool = False
    ) -> dict:
        """Process orthomosaic using MetaShape."""
        import Metashape
        
        # Configure GPU
        Metashape.app.gpu_mask = ~0
        
        # Setup paths
        output_path.mkdir(parents=True, exist_ok=True)
        project_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Check if project exists
        project_exists = project_path.exists()
        
        if project_exists and not clean_intermediate_files:
            logging.info(f"Loading existing project: {project_path}")
            doc = Metashape.Document()
            doc.open(str(project_path))
            if len(doc.chunks) > 0:
                chunk = doc.chunks[0]
            else:
                chunk = doc.addChunk()
        else:
            if clean_intermediate_files and project_exists:
                project_path.unlink()
            doc = Metashape.Document()
            doc.save(str(project_path))
            chunk = doc.addChunk()
        
        # Add photos
        if len(chunk.cameras) == 0:
            logging.info(f"Adding photos from: {photos_dir}")
            photos = list(photos_dir.glob("*.jpg")) + list(photos_dir.glob("*.JPG"))
            if not photos:
                raise ValueError(f"No images found in {photos_dir}")
            chunk.addPhotos([str(p) for p in photos])
            doc.save()
        
        # Add GCPs if requested
        if use_gcps and gcp_file and gcp_file.exists():
            if len(chunk.markers) == 0:
                logging.info(f"Loading GCPs from: {gcp_file}")
                chunk.importMarkers(str(gcp_file))
                doc.save()
        
        # Match photos
        if chunk.tie_points is None or len(chunk.tie_points) == 0:
            logging.info("Matching photos...")
            chunk.matchPhotos(
                downscale=photo_match_quality,
                tiepoint_limit=tiepoint_limit,
            )
            doc.save()
        
        # Align cameras
        aligned = sum(1 for cam in chunk.cameras if cam.transform)
        if aligned == 0:
            logging.info("Aligning cameras...")
            chunk.alignCameras()
            doc.save()
        
        # Build depth maps
        if len(chunk.depth_maps) == 0:
            logging.info("Building depth maps...")
            chunk.buildDepthMaps(
                downscale=depth_map_quality,
                filter_mode=Metashape.MildFiltering
            )
            doc.save()
        
        # Build model
        if chunk.model is None:
            logging.info("Building 3D model...")
            chunk.buildModel()
            doc.save()
        
        # Build orthomosaic
        if chunk.orthomosaic is None:
            logging.info("Building orthomosaic...")
            chunk.buildOrthomosaic()
            doc.save()
        
        # Export GeoTIFF
        ortho_path = output_path / f"{product_id}.tif"
        if not ortho_path.exists() or clean_intermediate_files:
            logging.info(f"Exporting GeoTIFF to: {ortho_path}")
            compression = Metashape.ImageCompression()
            compression.tiff_compression = Metashape.ImageCompression.TiffCompressionNone
            compression.tiff_big = True
            chunk.exportRaster(str(ortho_path), image_compression=compression)
            doc.save()
        
        stats = {
            'product_id': product_id,
            'use_gcps': use_gcps,
            'num_photos': len(chunk.cameras),
            'num_markers': len(chunk.markers),
            'ortho_path': str(ortho_path),
            'project_path': str(project_path),
        }
        
        return stats
    
    print("✓ MetaShape processing functions defined locally")
elif not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Cannot process orthomosaics.")
else:
    print("✓ Using MetaShape processor from qualicum_beach_gcp_analysis")


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Setup paths for Dataset 1
    dataset1_dir = data_dir / "DJI_202510060955_017_25-3288"
    intermediate_dir = output_dir / "intermediate"
    ortho_output_dir = output_dir / "orthomosaics"
    
    # Process orthomosaic WITHOUT GCPs
    print("=" * 60)
    print("Processing Dataset 1 - WITHOUT GCPs...")
    print("=" * 60)
    
    project_path1_no_gcps = intermediate_dir / "dataset1_no_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats1_no_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_no_gcps,
            product_id="dataset1_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    else:
        stats1_no_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_no_gcps,
            product_id="dataset1_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    
    print("\n✓ Dataset 1 processing (without GCPs) complete!")
    print(f"  Number of photos: {stats1_no_gcps['num_photos']}")
    print(f"  Orthomosaic: {stats1_no_gcps['ortho_path']}")
    
    ortho1_no_gcps_path = Path(stats1_no_gcps['ortho_path'])


## Step 6: Process Dataset 1 - WITH GCPs


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Process orthomosaic WITH GCPs
    print("=" * 60)
    print("Processing Dataset 1 - WITH GCPs...")
    print("=" * 60)
    
    project_path1_with_gcps = intermediate_dir / "dataset1_with_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats1_with_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset1_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    else:
        stats1_with_gcps = process_orthomosaic(
            photos_dir=dataset1_dir,
            output_path=ortho_output_dir,
            project_path=project_path1_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset1_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    
    print("\n✓ Dataset 1 processing (with GCPs) complete!")
    print(f"  Number of photos: {stats1_with_gcps['num_photos']}")
    print(f"  Number of markers: {stats1_with_gcps.get('num_markers', 0)}")
    print(f"  Orthomosaic: {stats1_with_gcps['ortho_path']}")
    
    ortho1_with_gcps_path = Path(stats1_with_gcps['ortho_path'])


## Step 7: Process Dataset 2 - WITHOUT GCPs


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Setup paths for Dataset 2
    dataset2_dir = data_dir / "DJI_202510060955_019_25-3288"
    
    # Process orthomosaic WITHOUT GCPs
    print("=" * 60)
    print("Processing Dataset 2 - WITHOUT GCPs...")
    print("=" * 60)
    
    project_path2_no_gcps = intermediate_dir / "dataset2_no_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats2_no_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_no_gcps,
            product_id="dataset2_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    else:
        stats2_no_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_no_gcps,
            product_id="dataset2_no_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=False
        )
    
    print("\n✓ Dataset 2 processing (without GCPs) complete!")
    print(f"  Number of photos: {stats2_no_gcps['num_photos']}")
    print(f"  Orthomosaic: {stats2_no_gcps['ortho_path']}")
    
    ortho2_no_gcps_path = Path(stats2_no_gcps['ortho_path'])


## Step 8: Process Dataset 2 - WITH GCPs


In [ ]:
if not METASHAPE_AVAILABLE:
    print("⚠️  MetaShape not available. Skipping processing.")
else:
    # Process orthomosaic WITH GCPs
    print("=" * 60)
    print("Processing Dataset 2 - WITH GCPs...")
    print("=" * 60)
    
    project_path2_with_gcps = intermediate_dir / "dataset2_with_gcps.psx"
    
    if USE_QUALICUM_PACKAGE:
        stats2_with_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset2_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    else:
        stats2_with_gcps = process_orthomosaic(
            photos_dir=dataset2_dir,
            output_path=ortho_output_dir,
            project_path=project_path2_with_gcps,
            gcp_file=gcp_file_for_processing,
            product_id="dataset2_with_gcps",
            clean_intermediate_files=False,
            photo_match_quality=PhotoMatchQuality.MediumQuality,
            depth_map_quality=DepthMapQuality.MediumQuality,
            tiepoint_limit=10000,
            use_gcps=True
        )
    
    print("\n✓ Dataset 2 processing (with GCPs) complete!")
    print(f"  Number of photos: {stats2_with_gcps['num_photos']}")
    print(f"  Number of markers: {stats2_with_gcps.get('num_markers', 0)}")
    print(f"  Orthomosaic: {stats2_with_gcps['ortho_path']}")
    
    ortho2_with_gcps_path = Path(stats2_with_gcps['ortho_path'])


## Step 9: Download Reference Basemap


In [ ]:
# Calculate bounding box from GCPs for basemap download
# Convert UTM bounds to lat/lon
center_easting = (min_y + max_y) / 2  # Y column is easting
center_northing = (min_x + max_x) / 2  # X column is northing
lat_center, lon_center = utm.to_latlon(center_easting, center_northing, 10, 'N')

# Convert bounds
lat_min, lon_min = utm.to_latlon(min_y, min_x, 10, 'N')
lat_max, lon_max = utm.to_latlon(max_y, max_x, 10, 'N')

# Add padding
padding = 0.001
bbox = (lat_min - padding, lon_min - padding, lat_max + padding, lon_max + padding)

print(f"Basemap bounding box: {bbox}")

# Download basemap
basemap_path = download_basemap(
    bbox=bbox,
    output_path=str(output_dir / "basemap.tif"),
    source="esri_world_imagery",
    target_resolution=0.1  # 0.1m per pixel
)

print(f"\n✓ Basemap saved to: {basemap_path}")


## Step 10: Compare Orthomosaics to Basemap


In [ ]:
# Compare all orthomosaics to basemap
comparison_dir = output_dir / "comparisons"
comparison_dir.mkdir(exist_ok=True)

# Determine orthomosaic paths
if 'ortho1_no_gcps_path' not in locals():
    ortho_output_dir = output_dir / "orthomosaics"
    ortho1_no_gcps_path = ortho_output_dir / "dataset1_no_gcps.tif"
    ortho1_with_gcps_path = ortho_output_dir / "dataset1_with_gcps.tif"
    ortho2_no_gcps_path = ortho_output_dir / "dataset2_no_gcps.tif"
    ortho2_with_gcps_path = ortho_output_dir / "dataset2_with_gcps.tif"

print("=" * 60)
print("Comparing orthomosaics to basemap...")
print("=" * 60)

# Compare Dataset 1 - Without GCPs
if ortho1_no_gcps_path.exists():
    print("\nDataset 1 - Without GCPs")
    print("-" * 60)
    metrics1_no_gcps = compare_orthomosaic_to_basemap(
        str(ortho1_no_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset1_no_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho1_no_gcps_path}")
    metrics1_no_gcps = {}

# Compare Dataset 1 - With GCPs
if ortho1_with_gcps_path.exists():
    print("\nDataset 1 - With GCPs")
    print("-" * 60)
    metrics1_with_gcps = compare_orthomosaic_to_basemap(
        str(ortho1_with_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset1_with_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho1_with_gcps_path}")
    metrics1_with_gcps = {}

# Compare Dataset 2 - Without GCPs
if ortho2_no_gcps_path.exists():
    print("\nDataset 2 - Without GCPs")
    print("-" * 60)
    metrics2_no_gcps = compare_orthomosaic_to_basemap(
        str(ortho2_no_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset2_no_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho2_no_gcps_path}")
    metrics2_no_gcps = {}

# Compare Dataset 2 - With GCPs
if ortho2_with_gcps_path.exists():
    print("\nDataset 2 - With GCPs")
    print("-" * 60)
    metrics2_with_gcps = compare_orthomosaic_to_basemap(
        str(ortho2_with_gcps_path),
        str(basemap_path),
        output_dir=str(comparison_dir / "dataset2_with_gcps")
    )
else:
    print(f"\n⚠️  Orthomosaic not found: {ortho2_with_gcps_path}")
    metrics2_with_gcps = {}

print("\n✓ All comparisons complete!")


## Step 11: Generate Quality Report


In [ ]:
# Create comparison summary
import pandas as pd

# Helper function to safely get metrics
def get_metric(metrics_dict, key, default=0.0):
    """Safely get a metric value, handling different key formats."""
    if not metrics_dict:
        return default
    # Try direct key
    if key in metrics_dict:
        return metrics_dict[key]
    # Try with _pixels suffix
    if f"{key}_pixels" in metrics_dict:
        return metrics_dict[f"{key}_pixels"]
    return default

# Collect metrics
summary_data = {
    'Dataset': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2'],
    'GCPs Used': ['No', 'Yes', 'No', 'Yes'],
    'RMSE': [
        get_metric(metrics1_no_gcps, 'rmse'),
        get_metric(metrics1_with_gcps, 'rmse'),
        get_metric(metrics2_no_gcps, 'rmse'),
        get_metric(metrics2_with_gcps, 'rmse'),
    ],
    'MAE': [
        get_metric(metrics1_no_gcps, 'mae'),
        get_metric(metrics1_with_gcps, 'mae'),
        get_metric(metrics2_no_gcps, 'mae'),
        get_metric(metrics2_with_gcps, 'mae'),
    ],
    'Correlation': [
        get_metric(metrics1_no_gcps, 'correlation'),
        get_metric(metrics1_with_gcps, 'correlation'),
        get_metric(metrics2_no_gcps, 'correlation'),
        get_metric(metrics2_with_gcps, 'correlation'),
    ],
    'SSIM': [
        get_metric(metrics1_no_gcps, 'ssim'),
        get_metric(metrics1_with_gcps, 'ssim'),
        get_metric(metrics2_no_gcps, 'ssim'),
        get_metric(metrics2_with_gcps, 'ssim'),
    ],
    'Displacement (pixels)': [
        get_metric(metrics1_no_gcps, 'displacement_magnitude'),
        get_metric(metrics1_with_gcps, 'displacement_magnitude'),
        get_metric(metrics2_no_gcps, 'displacement_magnitude'),
        get_metric(metrics2_with_gcps, 'displacement_magnitude'),
    ],
    'Num Matches': [
        get_metric(metrics1_no_gcps, 'num_matches', default=0),
        get_metric(metrics1_with_gcps, 'num_matches', default=0),
        get_metric(metrics2_no_gcps, 'num_matches', default=0),
        get_metric(metrics2_with_gcps, 'num_matches', default=0),
    ],
}

df = pd.DataFrame(summary_data)

print("\n" + "=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
print(df.to_string(index=False))

# Save summary to CSV
summary_csv = output_dir / "quality_summary.csv"
df.to_csv(summary_csv, index=False)
print(f"\n✓ Summary saved to: {summary_csv}")

# Create visualizations
available_metrics = []
for col in ['RMSE', 'MAE', 'Correlation', 'SSIM', 'Displacement (pixels)', 'Num Matches']:
    if col in df.columns and df[col].sum() > 0:
        available_metrics.append(col)

if available_metrics:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(available_metrics[:6]):
        ax = axes[i]
        df_pivot = df.pivot(index='Dataset', columns='GCPs Used', values=metric)
        df_pivot.plot(kind='bar', ax=ax, rot=0)
        ax.set_title(metric)
        ax.set_ylabel('Value')
        ax.legend(title='GCPs Used')
        ax.grid(True, alpha=0.3)
    
    # Hide unused subplots
    for i in range(len(available_metrics), 6):
        axes[i].axis('off')
    
    plt.tight_layout()
    report_plot_path = output_dir / "quality_report.png"
    plt.savefig(report_plot_path, dpi=150, bbox_inches='tight')
    print(f"✓ Quality report plot saved to: {report_plot_path}")
    plt.show()
else:
    print("⚠️  No metrics available for visualization")

# Generate markdown report
report_md = output_dir / "quality_report.md"
with open(report_md, 'w') as f:
    f.write("# Westminster Ground Truth Analysis - Quality Report\n\n")
    f.write("## Summary\n\n")
    f.write(df.to_markdown(index=False))
    f.write("\n\n## Findings\n\n")
    f.write("### Dataset 1\n")
    f.write(f"- **Without GCPs**: RMSE={get_metric(metrics1_no_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics1_no_gcps, 'displacement_magnitude'):.2f} pixels\n")
    f.write(f"- **With GCPs**: RMSE={get_metric(metrics1_with_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics1_with_gcps, 'displacement_magnitude'):.2f} pixels\n\n")
    f.write("### Dataset 2\n")
    f.write(f"- **Without GCPs**: RMSE={get_metric(metrics2_no_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics2_no_gcps, 'displacement_magnitude'):.2f} pixels\n")
    f.write(f"- **With GCPs**: RMSE={get_metric(metrics2_with_gcps, 'rmse'):.3f}, ")
    f.write(f"Displacement={get_metric(metrics2_with_gcps, 'displacement_magnitude'):.2f} pixels\n")

print(f"✓ Markdown report saved to: {report_md}")
